<a href="https://colab.research.google.com/github/aysedata/NLP/blob/main/NLP_3_(Word_Embedding_(word2vec%2C_Glove))(EU)_30_Jan_2023_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Word_embedding

Word embedding, bir kelime veya ifade için düz bir vektör olarak ifade edilmesidir. Bu vektörler, kelimelerin anlamları arasındaki benzerlikleri yansıtmak üzere öğrenilir. Bu sayede, yapay zeka modelleri kelimeler arasındaki ilişkileri daha iyi anlayabilir

Word embedding, metin verileri kullanılarak öğrenilir. Örneğin, bir metin corpus'u (büyük metin veritabanı) kullanılarak, her kelime için çevresindeki kelimeler ile ilişkisini öğrenir. Bu ilişki, kelime ve çevresindeki kelimeler arasındaki co-occurrence (ortak görünüm) frekansı olarak tanımlanır. Daha sonra, bu co-occurrence verileri kullanarak, her kelime için bir vektör oluşturulur. Bu vektörler, kelimelerin anlamları arasındaki benzerlikleri yansıtmak üzere tasarlanır.

Örneğin, "king" ve "queen" kelimeleri sıklıkla aynı metinlerde görülür ve benzer anlamlar taşır, bu nedenle oluşan vektörleri birbirine yakın olacak. Ancak "king" ve "orange" kelimeleri sıklıkla aynı metinlerde görülmez ve farklı anlamlar taşır, bu nedenle oluşan vektörleri birbirinden uzak olacak.

## Feature Representation (Feature Extraction for word embeddings)

word embbeding vektöründe yeralan ve genellikle -1 ve +1 (Bazı pre-trained modellerde bu değer -1 ve +1'den büyük olabilir.) arasında değer alan elemanların her birine feature representation denir. Bu feature representationların model tarafından oluşturulması sürecisine ise feature extraction denir.

ML'de, feature extraction, veri kümesinde bulunan özellikleri veya nitelikleri belirlemek için kullanılan bir yöntemdir. Bu özellikler, veri kümesindeki verileri anlamlı bir şekilde ifade etmelerine olanak tanır. Bu özellikler, veri kümesinden manuel olarak seçilir.

DL'de ise, feature extraction, veri kümesindeki özellikleri öğrenmek için kullanılan bir yöntemdir. Bu yöntem için kullanılan yöntemler arasında Convolutional Neural Network (CNN) ve Recurrent Neural Network (RNN) gibi yapay sinir ağları yer almaktadır. Bu yapay sinir ağları, veri kümesinden özellikleri otomatik olarak öğrenir ve bu özellikler, veri kümesindeki verileri daha anlamlı bir şekilde ifade etmelerine olanak tanır.

Özet olarak, ML'de özellikler manuel olarak kullanıcı tarafından seçilirken, DL'de özellikler otomatik model tarafından tespit edilerek öğrenilir.

## Word2Vec

In [1]:
# pip install gensim
# conda install -c conda-forge gensim

# gensim kütüphanesi anaconda'da default yüklü olarak geliyor.

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from nltk.tokenize import word_tokenize
import pandas as pd
from gensim.models import Word2Vec

import warnings
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv('newspaper.zip', names = ["news"])
df

# pandas'ın read_csv fonksiyonu zip'li dosyaları da okuyabiliyor. 
# ziplediğimiz data(corpus) txt dosyası olduğundan dosyada belirlenmiş feture ismi/isimleri belirlenmemiştir. Bu sebeple 
# feature isimlendirmesi için names parametresini kullanıyoruz.
# names parametresi kullanılmaz ise ilk satırdaki text default olarak feature names olarak atanır.
#noktalama isaretleri temizlenmis, hepsi kucuk harf.
# STOP WORD'LERI TEMIZLEME!! deep model ogrenecek.

,news
0,iran devlet televizyonu ülkedeki eyaletin sind...
1,gösterilerde fitnecilere ölüm münafıklara ölüm...
2,dini lider ali hamaney ve cumhurbaşkanı mahmud...
3,musevi ye ölüm ve idam idam sloganları duyuldu
4,muhalefet liderleri kaçtı mı aşure günü yaşana...
...,...
411520,dışişleri bakanlığı ndan yapılan yazılı açıkla...
411521,açıklamada abd nin ankara büyükelçiliği ve ist...
411522,seyahat uyarısı güncelleme kararının temmuz da...
411523,amerikalı turistlerin açıkça türkiye deki ulus...


In [5]:
df.news[0]

'iran devlet televizyonu ülkedeki eyaletin sinde yapılan reformcuları protesto amaçlı yürüyüşlere milyonlarca kişinin katıldığını bildirdi '

In [8]:
word_tokenize(df.news[0])

['iran',
 'devlet',
 'televizyonu',
 'ülkedeki',
 'eyaletin',
 'sinde',
 'yapılan',
 'reformcuları',
 'protesto',
 'amaçlı',
 'yürüyüşlere',
 'milyonlarca',
 'kişinin',
 'katıldığını',
 'bildirdi']

In [9]:
corpus = []

for i in df.index:
    corpus.append(word_tokenize(df.news[i]))

print(corpus[:5])

# word2vec algoritması tüm corpusun 2 boyutlu olmasını ister. Bu sebeple burdaki for döngüsü ile tüm documnetleri/satırları
# teker teker çekip word tokenlerine ayırıyoruz. Word_tokenize fonksiyonu default olarak texti word tokenlerine ayırıp bir 
# listeye atar. Bu listeler corpus listesine append edilerek corpus 2 boyutlu hale getiritir. 

[['iran', 'devlet', 'televizyonu', 'ülkedeki', 'eyaletin', 'sinde', 'yapılan', 'reformcuları', 'protesto', 'amaçlı', 'yürüyüşlere', 'milyonlarca', 'kişinin', 'katıldığını', 'bildirdi'], ['gösterilerde', 'fitnecilere', 'ölüm', 'münafıklara', 'ölüm', 'abd', 'ye', 'ölüm', 'ingiltere', 'ye', 'ölüm', 'sloganları', 'atıldı'], ['dini', 'lider', 'ali', 'hamaney', 've', 'cumhurbaşkanı', 'mahmud', 'ahmedinejad', 'ı', 'destekleyen', 'iranlılar', 'son', 'olaylarda', 'yeğeni', 'öldürülen', 'mir', 'hüseyin', 'musevi', 'başta', 'olmak', 'üzere', 'muhalefet', 'liderlerini', 'kınadılar'], ['musevi', 'ye', 'ölüm', 've', 'idam', 'idam', 'sloganları', 'duyuldu'], ['muhalefet', 'liderleri', 'kaçtı', 'mı', 'aşure', 'günü', 'yaşanan', 'çatışmalarda', 'devlet', 'kaynaklarına', 'göre', 'u', 'terörist', 'olmak', 'üzere', 'kişi', 'ölmüştü']]


In [ ]:
model = Word2Vec(corpus, vector_size=100, window=5, min_count=5, sg=1)

# vector_size, kaç boyutlu word embeddings istiyorsak burda belirtiyoruz.
# Bir token ile diğer tokenler arasındaki anlamsal ilişkiler kurulurken bu tokenin kaç önce ve sonrasındaki tokenler dikkate 
# alınarak eğitim yapılmasını istiyorsak window parametresinde belirtiyoruz. Tavsiye edilen 5-15 sayıları arasındadır.
# min_count, corpusta 5 veya daha az geçen tokenler eğitime dahil edilmez. Genellikle 3,4,5 gibi sayılar tercih edilir.
# sg =1, eğitimi skipgram algoritması ile yap.
# sg =0, eğitimi CBOW algoritması ile yap.

In [ ]:
model.wv['ankara']

# 100 elemanlı/boyutlu word_embedding

array([ 0.3441693 ,  0.18543379, -0.19872634,  0.23503837,  0.16469045,
        0.2865687 , -0.2889401 ,  0.9429464 , -0.2804621 , -0.33941638,
       -0.21067587, -0.16293707, -0.25710574, -0.11478771, -0.0072794 ,
        0.6861819 ,  0.13681021, -0.13114394,  0.07727321, -1.009751  ,
       -0.06335889,  0.23829187,  0.6200752 , -0.39470533,  0.04591637,
        0.28344986, -0.07981466,  0.30331495, -0.23600689,  0.4277663 ,
        0.2816101 ,  0.5380762 , -0.22863017, -0.8429164 ,  0.15260535,
       -0.14532804,  0.05964754, -0.2269461 ,  0.2783678 , -0.27150017,
        0.5624108 ,  0.06899057,  0.32698855, -0.03078212,  0.1455206 ,
        0.38612798, -0.423533  ,  0.09521478,  0.36564443, -0.44070894,
        0.21834393, -0.16483161, -0.05296769, -0.01286471, -0.03060687,
       -0.5058217 , -0.00110918, -0.5460528 , -0.558082  , -0.22550768,
       -0.03117701, -0.24768427,  0.1458736 ,  0.06167162,  0.04401443,
        0.11583218,  0.5002228 ,  0.2951722 ,  0.12509935, -0.23

In [ ]:
model.wv.most_similar('öğretmen')

[('okuldaki', 0.7611968517303467),
 ('öğretmeni', 0.7440207600593567),
 ('öğrencisinin', 0.7435778975486755),
 ('öğretmenin', 0.7422101497650146),
 ('öğrencilerine', 0.7287434339523315),
 ('öğrenci', 0.7251050472259521),
 ('erkek', 0.7167589068412781),
 ('öğrencinin', 0.7128359079360962),
 ('öğretmenlik', 0.7113512754440308),
 ('üniversite', 0.7061052918434143)]

In [ ]:
model.wv.most_similar('kırmızı')

[('çizgileri', 0.6992867588996887),
 ('gömlekli', 0.6750586032867432),
 ('bülten', 0.6317101716995239),
 ('ışıkta', 0.6302773952484131),
 ('gömlekliler', 0.6299250721931458),
 ('turuncu', 0.6290749311447144),
 ('sarı', 0.6280034184455872),
 ('halıda', 0.6223219633102417),
 ('bültenle', 0.6065525412559509),
 ('halı', 0.600972056388855)]

In [ ]:
model.wv.most_similar('eve')

[('evine', 0.8428974151611328),
 ('dükkana', 0.784963071346283),
 ('apartmana', 0.7660619020462036),
 ('mağazaya', 0.7406488060951233),
 ('arabaya', 0.73406982421875),
 ('karakola', 0.7220742106437683),
 ('otobüse', 0.7144497632980347),
 ('odasına', 0.7138554453849792),
 ('evden', 0.7110965251922607),
 ('arabasına', 0.7104228734970093)]

In [ ]:
model.wv.most_similar('mavi')

[('marmara', 0.8910354375839233),
 ('gemisine', 0.649941623210907),
 ('baskınıyla', 0.6298705339431763),
 ('filosundaki', 0.6043975949287415),
 ('saldırısındaki', 0.5984426140785217),
 ('baskınına', 0.5959968566894531),
 ('gemilere', 0.5947985649108887),
 ('dökme', 0.5919091701507568),
 ('gemisindeki', 0.5863186717033386),
 ('filodaki', 0.5833677649497986)]

In [ ]:
model.wv.most_similar(positive=['öğrenme', 'doktor'], negative=['tedavi'], topn=5)

[('psikoloji', 0.661553144454956),
 ('driscoll', 0.6602360010147095),
 ('yaptıklarında', 0.6546162366867065),
 ('tanıtıp', 0.6528038382530212),
 ('almancayı', 0.6497054100036621)]

In [ ]:
model.wv.most_similar(positive=['ankara', 'belçika'], negative=['brüksel'], topn=1)

[('hollanda', 0.688507080078125)]

In [ ]:
model.save("word2vec.model")

In [ ]:
model = Word2Vec.load("word2vec.model")

## Glove

In [ ]:
from gensim.models import KeyedVectors

# KeyedVectors fonksiyonunu farklı bir formattaki word embeddingleri word2vec formatına dönüştürmek için kullanıyoruz.

In [ ]:
glove_model = 'glove.6B.100d.txt'
model2 = KeyedVectors.load_word2vec_format(glove_model, no_header=True) #'glove.6B.100d.txt'

# 'glove.6B.100d.txt' word embedding textinde (vocab_size, vector_size) boyutuna ilişkin bir bildirim olmadığından KeyedVectors
# dönüştürücüsünün tüm texti tarayıp (vocab_size, vector_size) boyutunu tespit etmesi ve buna göre ilgili token/kelimelere
# karşılık gelen vektörleri döndürmesi gerekir. Bunu yapabilmesi için no_header=True olarak düzenleniyor.

In [ ]:
model2['teacher']

array([ 0.44374 ,  0.67311 , -0.51096 ,  0.20882 , -0.10662 ,  0.55098 ,
       -0.035593,  0.25126 , -0.32789 ,  1.0762  , -0.49637 , -0.4298  ,
        0.36764 ,  0.57894 , -0.25027 , -0.41021 ,  0.086998, -0.16843 ,
       -0.85764 ,  1.0404  , -1.0314  ,  0.095147,  0.30729 ,  0.12348 ,
        0.22745 , -0.52157 , -0.72478 , -1.0843  ,  0.035966,  0.62985 ,
       -1.0991  ,  0.67161 ,  0.33797 ,  0.14551 , -0.90049 , -0.064415,
       -0.75247 ,  0.21741 ,  0.51594 , -0.46291 , -0.77598 ,  0.40705 ,
        0.1889  , -0.43402 ,  0.23202 , -0.081453, -0.3882  , -0.34444 ,
        0.080225, -0.28274 , -0.38869 , -0.58152 , -0.25558 ,  1.0027  ,
       -0.11114 , -1.5402  , -0.16761 , -0.26558 ,  0.9325  ,  0.069397,
        0.96618 ,  0.15449 , -0.22905 , -0.1761  ,  0.13225 , -0.55741 ,
        0.9234  , -0.04845 ,  0.50202 ,  1.0144  , -0.1256  ,  0.30486 ,
        0.090808,  0.17642 , -0.23146 ,  0.68386 ,  0.37269 , -0.37316 ,
       -0.025728, -1.0279  , -0.33142 ,  0.036028, 

In [ ]:
model2.most_similar('ankara')

[('turkey', 0.7512096166610718),
 ('istanbul', 0.6787630319595337),
 ('turkish', 0.6690374612808228),
 ('damascus', 0.6372509002685547),
 ('tbilisi', 0.6322181820869446),
 ('erdogan', 0.6258037090301514),
 ('moscow', 0.6217040419578552),
 ('brussels', 0.6181437969207764),
 ('skopje', 0.6164302229881287),
 ('cyprus', 0.606403112411499)]

In [ ]:
model2.most_similar('teacher')

[('student', 0.8083398938179016),
 ('school', 0.7545564770698547),
 ('teaching', 0.7521439790725708),
 ('taught', 0.741184651851654),
 ('teachers', 0.7291542887687683),
 ('graduate', 0.7134961485862732),
 ('instructor', 0.7077120542526245),
 ('students', 0.6828974485397339),
 ('teaches', 0.6552315354347229),
 ('education', 0.6528989672660828)]

In [ ]:
model2.most_similar('doctor')

[('physician', 0.7673240303993225),
 ('nurse', 0.75215083360672),
 ('dr.', 0.7175194025039673),
 ('doctors', 0.7080884575843811),
 ('patient', 0.7074184417724609),
 ('medical', 0.6995992660522461),
 ('surgeon', 0.6905338168144226),
 ('hospital', 0.6900930404663086),
 ('psychiatrist', 0.658909797668457),
 ('dentist', 0.6447421312332153)]

In [ ]:
model2.most_similar(positive=['woman', 'son'], negative=['man'], topn=1)

[('daughter', 0.9090957641601562)]

In [ ]:
model2.most_similar(positive=['woman', 'father'], negative=['man'], topn=1)

[('mother', 0.9024619460105896)]

In [ ]:
model2.most_similar(positive=['woman', 'uncle'], negative=['man'], topn=1)

[('aunt', 0.8368030190467834)]

In [ ]:
model2.most_similar(positive=['ankara', 'germany'], negative=['berlin'], topn=1)

[('turkey', 0.81471186876297)]

In [ ]:
model2.most_similar(positive=['teach', 'doctor'], negative=['treat'], topn=1)

[('teacher', 0.7610154151916504)]

In [ ]:
model2.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)

[('queen', 0.7698540687561035)]

In [ ]:
model2.most_similar(positive=['love', 'jealous'], negative=['hate'], topn=1)

[('lover', 0.7032662630081177)]